# Regularization

Let's improve our understanding of what impacted **Titanic** passengers' chance of survival
- We will use logistic classifiers which are easy to interpret
- Remember we already did it with statsmodels in lecture "Decision Science - Logistic Regression"
- We were using `p-values` & statistical assumptions to detect which features were irrelevant / don't generalize
- This time, we will use `regularization` to detect relevant/irrelevant features based on under/overfitting criteria
- **Our goal is to compare `L1` and `L2` penalties**

## 1. We load and preprocess the data for you

In [29]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [30]:
data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/ML_titanic_dataset_encoded.csv")

# the dataset is already one-hot-encoded
data.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [31]:
# We build X and y

y = data["survived"]
X = data.drop(columns="survived")
X.head()

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [32]:
# We MinMaxScale our features for you
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X.shape

(714, 12)

## 1.  Logistic Regression without regularization

❓ Rank the features by decreasing order of importance after training a simple **non-regularized** Logistic Regression (i.e. look at the coefficients after fitting)
- Careful: `LogisticRegression` is penalized by default
  - take a look at the [penalty parameter](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) to find out how to remove the penalty)
- Increase `max_iter` to a larger number until model converges

<details>
    <summary>Hint</summary>
    <img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/05-ML/05-Model-Tuning/model_selection.png" alt="penalizing a regression" width="500" height="600">
</details>

In [33]:
log_reg = LogisticRegression(penalty=None)
log_reg.fit(X_scaled, y)
log_res = {
    "features":log_reg.feature_names_in_,
    "coefficients":log_reg.coef_[0]
}

features_coef_df = pd.DataFrame(log_res)
features_coef_df.sort_values(by=["coefficients"], ascending=False)

,features,coefficients
5,sex_female,2.680469
4,fare,1.443161
8,who_child,1.358557
6,class_First,0.738188
9,embark_town_Cherbourg,0.103875
11,embark_town_Southampton,-0.193365
10,embark_town_Queenstown,-0.580103
0,pclass,-0.693957
7,class_Third,-0.829465
3,parch,-0.925921


In [28]:
log_reg

LogisticRegression(penalty=None)

❓How do you interpret, in plain English, the value for the coefficient `sex_female`?

<details>
    <summary>Answer</summary>

> "All other things being equal (such as age, ticket class etc...),
being a women increases your log-odds of survival by 2.67 (your coef value)"
    
> "Controling for all other explaining factors available in this dataset,
being a women increases your odds of survival by exp(2.67) = 14"

</details>


❓ What is the feature that most impacts the chances of survival according to your model?  
Fill the `top_1_feature` list below with the name of this feature

In [19]:
top_1_feature = [""]

In [20]:
from nbresult import ChallengeResult
result = ChallengeResult('unregularized', top_1_feature = top_1_feature)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/branchedelac/.pyenv/versions/3.10.6/envs/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/branchedelac/code/branchedelac/05-ML/05-Model-Tuning/data-regularization/tests
plugins: anyio-3.6.2, asyncio-0.19.0, typeguard-2.13.3
asyncio: mode=strict
collecting ... collected 1 item

test_unregularized.py::TestUnregularized::test_top_1 FAILED              [100%]

=================================== FAILURES ===================================
_________________________ TestUnregularized.test_top_1 _________________________

self = <tests.test_unregularized.TestUnregularized testMethod=test_top_1>

    def test_top_1(self):
        res_set = set(self.result.top_1_feature)
        minimal_truth_set = {"embark_town_Southampton", "embark_town_Queenstown", "embark_town_Cherbourg"}
>       self.assertTrue(len(res_set & minimal_truth_set) == 

## 2.  Logistic Regression with an L2 penalty

Let's use a **Logistic model** whose log-loss has been penalized with a **L2** term to figure out the **most important features** without overfitting.  
This is the "classification" equivalent to the "Ridge" regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its features by importance (look at the coefficients)
- By "strongly regularized" we mean "more than Sklearn's default regularization factor". 
- Sklearn's default values are very useful orders of magnitudes to keep in mind for "scaled features"

In [21]:
# YOUR CODE HERE

❓ What are the top 2 features driving chances of survival according to your model?  
Fill the `top_2_features` list below with the name of these features

In [22]:
top_2_features = ["", ""]

#### 🧪 Test your code below

In [23]:
from nbresult import ChallengeResult
result = ChallengeResult('ridge', top_2 = top_2_features)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/branchedelac/.pyenv/versions/3.10.6/envs/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/branchedelac/code/branchedelac/05-ML/05-Model-Tuning/data-regularization/tests
plugins: anyio-3.6.2, asyncio-0.19.0, typeguard-2.13.3
asyncio: mode=strict
collecting ... collected 1 item

test_ridge.py::TestRidge::test_top2 FAILED                               [100%]

=================================== FAILURES ===================================
_____________________________ TestRidge.test_top2 ______________________________

self = <tests.test_ridge.TestRidge testMethod=test_top2>

    def test_top2(self):
        res_set = set(self.result.top_2)
        truth_set = {'sex_female', 'pclass', 'who_child', 'class_Third'}
>       self.assertTrue(len(res_set & truth_set) == 2, "Have you regularized enough? Notice that C = 1/alpha")
E       A

## 2. Logistic Regression with an L1 penalty

This time, we'll use a logistic model whose log-loss has been penalized with a **L1** term to **filter-out the less important features**.  
This is the "classification" equivalent to the **Lasso** regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its features by importance

In [24]:
# YOUR CODE HERE

❓ What are the features that have absolutely no impact on chances of survival, according to your L1 model?  
Fill the `zero_impact_features` list below with the name of these features; you may have to add elements to the list.

- Do you notice how some of them were "highly important" according to the non-regularized model? 
- From now on, we will always regularize our linear models!

In [25]:
zero_impact_features = ["", "", "", ""]

#### 🧪 Test your code below

In [26]:
from nbresult import ChallengeResult
result = ChallengeResult('lasso', zero_impact_features = zero_impact_features)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/branchedelac/.pyenv/versions/3.10.6/envs/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/branchedelac/code/branchedelac/05-ML/05-Model-Tuning/data-regularization/tests
plugins: anyio-3.6.2, asyncio-0.19.0, typeguard-2.13.3
asyncio: mode=strict
collecting ... collected 1 item

test_lasso.py::TestLasso::test_zero_impact FAILED                        [100%]

=================================== FAILURES ===================================
__________________________ TestLasso.test_zero_impact __________________________

self = <tests.test_lasso.TestLasso testMethod=test_zero_impact>

    def test_zero_impact(self):
        res_set = set(self.result.zero_impact_features)
        minimal_truth_set = {"embark_town_Queenstown"}
>       self.assertTrue(len(res_set & minimal_truth_set) == 1, "Have you regularized enough?")
E       Assert

**🏁 Congratulation! Don't forget to commit and push your notebook**